# Segmenting and Clustering Neighborhoods in Toronto

#### Begin by using pandas to read in table

In [1]:
import pandas as pd
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
df = table[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### get rid of "Not Assigned" neighborhoods

In [2]:
dropneighborhood = []
for postcodes in range(0,len(df)):
    if df.iloc[postcodes,1]=="Not assigned":
        dropneighborhood.append(postcodes)
    elif df.iloc[postcodes,2]=="Not assigned":
        df.iloc[postcodes,2]=df.iloc[postcodes,1]
df = df.drop(dropneighborhood)    
df = df.reset_index()
df = df.drop(columns=['index'])
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### group neighborhoods together by zip code

In [3]:
newdf = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
newdf.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [4]:
newdf.shape

(103, 3)

In [51]:
newdf.iloc[postcodes,0]+" "+newdf.iloc[postcodes,1]+" "+newdf.iloc[postcodes,2]

'M1C Scarborough Highland Creek,Rouge Hill,Port Union'

In [38]:
import geocoder
latitude = []
longitude =[]

for postcodes in range(len(newdf)):
    tests = newdf.iloc[postcodes,2].split(',')
    first_hood = test[0]
    location = geocoder.arcgis(first_hood+" "+newdf.iloc[postcodes,1]+" "+newdf.iloc[postcodes,0])
    location.latlng

    # loop until you get the coordinates
    latitude.append(location.latlng[0])
    longitude.append(location.latlng[1])
    
newdf['Latitude'] = latitude
newdf['Longitude'] = longitude
newdf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
